# Load Data

In [8]:
%reload_ext autoreload
%autoreload 2

In [9]:
from result_saver import SaverProvider
provider = SaverProvider()

In [10]:
import numpy as np
from Scratch import metadata_loader

DEVICE = 'ibm_cusco'

md = metadata_loader(_extract=True, _drop_inutile=True)
md = md[md["job_status"] == "JobStatus.DONE"]
md = md[md["notebook_name"] == "diff_rounds_fixed_dist"]
max_distance = int(max(md.distance))
max_distance = 55
md = md[md["distance"] == max_distance]
md = md[md["backend_name"]==DEVICE]
md = md.sort_values(by=['rounds'], ascending=False)
max_rounds = '45'
md = md[md["rounds"] == max_rounds]

md = md[:]

md

,creation_date,notebook_name,backend_name,job_id,tags,shots,tags_xp,sampled_state,num_qubits,job_status,extra,optimization_level,code,distance,rounds,logical,layout,descr
1015,2023-11-03 09:43:01.431495+01:00,diff_rounds_fixed_dist,ibm_cusco,cn2b55c3r3vg008f53fg,[],330.0,NaN,NaN,NaN,JobStatus.DONE,NaN,NaN,RepetitionCodeCircuit,55.0,45,0,_is_hex=True,NaN
1016,2023-11-03 09:43:32.644931+01:00,diff_rounds_fixed_dist,ibm_cusco,cn2b5d5vcq70008qm4c0,[],330.0,NaN,NaN,NaN,JobStatus.DONE,NaN,NaN,RepetitionCodeCircuit,55.0,45,1,_is_hex=True,NaN


In [30]:
from soft_info import get_repcode_layout, get_KDEs

memories = {}
for job_id, logical in zip(md.job_id, md.logical):
    mmr_name = f"mmr_log_{logical}"
    memories[mmr_name] = provider.retrieve_job(job_id).result().get_memory()

memory = memories["mmr_log_0"][:]

layout = get_repcode_layout(distance=max_distance, backend=provider.get_backend(DEVICE), _is_hex=True)
kde_dict, scaler_dict = get_KDEs(provider, DEVICE, layout, bandwidths=0.2, plot=False)

In [28]:
from soft_info import find_longest_path_in_hex
print(layout)
hex_backend = provider.get_backend('ibm_sherbrooke')
path, _, _  = find_longest_path_in_hex(hex_backend)
distance = 55
bounded_path = path[:2 * distance - 1]
layout_v2 = bounded_path[1::2] + bounded_path[::2] 
print(layout)

[114, 116, 118, 120, 122, 104, 102, 100, 98, 96, 94, 75, 77, 79, 81, 83, 85, 66, 64, 62, 60, 58, 56, 37, 39, 41, 43, 45, 47, 28, 26, 24, 22, 20, 18, 0, 2, 4, 6, 8, 10, 12, 30, 32, 51, 49, 68, 70, 89, 87, 106, 108, 126, 124, 113, 115, 117, 119, 121, 111, 103, 101, 99, 97, 95, 90, 76, 78, 80, 82, 84, 73, 65, 63, 61, 59, 57, 52, 38, 40, 42, 44, 46, 35, 27, 25, 23, 21, 19, 14, 1, 3, 5, 7, 9, 11, 17, 31, 36, 50, 55, 69, 74, 88, 93, 107, 112, 125, 123]
[114, 116, 118, 120, 122, 104, 102, 100, 98, 96, 94, 75, 77, 79, 81, 83, 85, 66, 64, 62, 60, 58, 56, 37, 39, 41, 43, 45, 47, 28, 26, 24, 22, 20, 18, 0, 2, 4, 6, 8, 10, 12, 30, 32, 51, 49, 68, 70, 89, 87, 106, 108, 126, 124, 113, 115, 117, 119, 121, 111, 103, 101, 99, 97, 95, 90, 76, 78, 80, 82, 84, 73, 65, 63, 61, 59, 57, 52, 38, 40, 42, 44, 46, 35, 27, 25, 23, 21, 19, 14, 1, 3, 5, 7, 9, 11, 17, 31, 36, 50, 55, 69, 74, 88, 93, 107, 112, 125, 123]


# Decoding

In [31]:
import stim
import pymatching

d = max_distance
T = int(max_rounds)

circuit = stim.Circuit.generated("repetition_code:memory",
                                 distance=d,
                                 rounds=T,
                                 after_clifford_depolarization=0.1)

model = circuit.detector_error_model(decompose_errors=True)
matching = pymatching.Matching.from_detector_error_model(model)


In [32]:
from tqdm import tqdm
from soft_info import get_counts, soft_reweight_pymatching, counts_to_det_syndr, draw_matching_graph, reweight_edges_to_one

VERBOSE = False

#actual_observables = np.array([[False]]) # hardcoded, can be retrieved
num_errors = 0

i = 0
w_idx_lst = []
for shot in tqdm(range(len(memory))[:]):
    i += 1
    IQ_data = memory[shot]

    counts = get_counts([IQ_data], kde_dict, scaler_dict, layout, T, verbose=False)
    count_key = next(iter(counts.keys()))
    
    actual_observables = [(int(count_key[0])+0)%2]
    soft_reweight_pymatching(matching, d, T, IQ_data, kde_dict, layout, scaler_dict, common_measure=0.01, verbose=False)  
    #reweight_edges_to_one(matching)

    array_processed_string = counts_to_det_syndr(count_key, _resets=False, verbose=False)

    predicted_observables = matching.decode(array_processed_string)

    if predicted_observables == actual_observables: #== [0]:
        continue
    num_errors += 1
    #print("actual_observables:", actual_observables)
    #print("predicted_observables:", predicted_observables)
    print("num_errors:", num_errors)

    #print(f"Wrong decoding at index {i}")
    w_idx_lst.append(i)

    
    if VERBOSE:
        print("Count key:", count_key)
        print("count_string_to_syndromes:", array_processed_string)
        print("actual_observables:", actual_observables)
        print("predicted_observables:", predicted_observables)

    
    if VERBOSE:
        matched_edges = matching.decode_to_edges_array(array_processed_string)
        print("matched_edges: ", matched_edges)
        print("Estimated flip:", predicted_observables)
        
    if VERBOSE:
        draw_matching_graph(matching, d, T, syndromes=array_processed_string, matched_edges=matched_edges, figsize=(10, 10))


    

print("Num errors:", num_errors)

  1%|          | 4/330 [00:21<29:33,  5.44s/it]

num_errors: 1


  5%|▍         | 15/330 [01:21<27:43,  5.28s/it]

num_errors: 2


 10%|▉         | 32/330 [02:51<26:21,  5.31s/it]

...

num_errors: 61
num_errors: 62


 99%|█████████▉| 328/330 [28:47<00:10,  5.09s/it]

num_errors: 63


100%|██████████| 330/330 [28:57<00:00,  5.26s/it]

Num errors: 63


In [33]:
from tqdm import tqdm
from soft_info import get_counts, soft_reweight_pymatching, counts_to_det_syndr, draw_matching_graph, reweight_edges_to_one

VERBOSE = False

#actual_observables = np.array([[False]]) # hardcoded, can be retrieved
num_errors = 0

i = 0
w_idx_lst = []
for shot in tqdm(range(len(memory))[:]):
    i += 1
    IQ_data = memory[shot]

    counts = get_counts([IQ_data], kde_dict, scaler_dict, layout, T, verbose=False)
    count_key = next(iter(counts.keys()))
    
    actual_observables = [(int(count_key[0])+0)%2]
    #soft_reweight_pymatching(matching, d, T, IQ_data, kde_dict, layout, scaler_dict, common_measure=0.01, verbose=False)  
    reweight_edges_to_one(matching)

    array_processed_string = counts_to_det_syndr(count_key, _resets=False, verbose=False)

    predicted_observables = matching.decode(array_processed_string)

    if predicted_observables == actual_observables: #== [0]:
        continue
    num_errors += 1
    #print("actual_observables:", actual_observables)
    #print("predicted_observables:", predicted_observables)
    print("num_errors:", num_errors)

    #print(f"Wrong decoding at index {i}")
    w_idx_lst.append(i)

    
    if VERBOSE:
        print("Count key:", count_key)
        print("count_string_to_syndromes:", array_processed_string)
        print("actual_observables:", actual_observables)
        print("predicted_observables:", predicted_observables)

    
    if VERBOSE:
        matched_edges = matching.decode_to_edges_array(array_processed_string)
        print("matched_edges: ", matched_edges)
        print("Estimated flip:", predicted_observables)
        
    if VERBOSE:
        draw_matching_graph(matching, d, T, syndromes=array_processed_string, matched_edges=matched_edges, figsize=(10, 10))


    

print("Num errors:", num_errors)

  1%|          | 4/330 [00:07<09:50,  1.81s/it]

num_errors: 1
num_errors: 2


  2%|▏         | 5/330 [00:09<09:44,  1.80s/it]

num_errors: 3


  3%|▎         | 10/330 [00:18<09:42,  1.82s/it]

...

num_errors: 110
num_errors: 111


 99%|█████████▉| 328/330 [09:54<00:03,  1.77s/it]

num_errors: 112


100%|██████████| 330/330 [09:58<00:00,  1.81s/it]

Num errors: 112
